In [21]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Charger le dataset des films

movies = pd.read_csv(
    "../data/movies.csv"
)

ratings = pd.read_csv("../data/ratings.csv")
ratings = ratings.drop(columns=["timestamp"])

# Afficher les premières lignes de chaque table pour voir leur contenu

movies["genres"] = movies["genres"].str.split("|")  # Divise les genres en listes

# Construire la matrice utilisateur-film

user_movie_matrix = ratings.pivot(index="userId", columns="movieId", values="rating")

# Remplir les valeurs manquantes par 0 (aucune note donnée)

user_movie_matrix = user_movie_matrix.fillna(0)

# Calculer la similarité cosinus entre utilisateurs

user_similarity_matrix = cosine_similarity(user_movie_matrix)

# Convertir en DataFrame pour plus de lisibilité

user_similarity_df = pd.DataFrame(user_similarity_matrix, index=user_movie_matrix.index, columns=user_movie_matrix.index)

print("Matrice de similarité entre utilisateurs calculée :")
print(user_similarity_df.head())

all_genres = set(
    g for genres_list in movies["genres"] for g in genres_list
)  # Liste unique de tous les genres

# Sélection des colonnes de genres uniquement

genre_columns = list(all_genres)

# Calcul de la similarité cosinus
# similarity_matrix = cosine_similarity(genre_data)

print("Matrice de similarité entre films calculée.")

# Ajouter une colonne par genre et mettre 1 si le film appartient au genre, 0 sinon

for genre in all_genres:
    movies[genre] = movies["genres"].apply(lambda genres: 1 if genre in genres else 0)
print("Movies dataset with genres transformed:")

# Maintenant, on peut créer genre_data et calculer la similarité

genre_columns = list(all_genres)
genre_data = movies[genre_columns]

# Choisir deux films par leurs indices, par exemple les deux premiers films
# Utilise leurs vecteurs de genres en sélectionnant les lignes correspondantes

film1_id = 0  # Index du premier film (peut aussi être un autre index)
# film2_id = 1  # Index du deuxième film

# Extraire les vecteurs de genres pour les deux films

# film1_vector = genre_data.iloc[film1_id].values.reshape(1, -1)
# film2_vector = genre_data.iloc[film2_id].values.reshape(1, -1)

# Calcul de la similarité cosinus entre les deux films

# similarity_score = cosine_similarity(film1_vector, film2_vector)[0][0]

similarity_score = cosine_similarity(genre_data)

# Afficher un aperçu de la matrice de similarité

print("Matrice de similarité calculée entre tous les films.")
# print(similarity_score[:5, :5])  # Affiche les 5 premières lignes et colonnes pour un aperçu

# print(f"Similarité entre le film {film1_id} et le film {film2_id} : {similarity_score}")
def recommend_movies(movie_index, similarity_score, movies, num_recommendations=5, exclude_movie = None):
    
    # Obtenir les scores de similarité pour le film spécifié
    similarity_scores = list(enumerate(similarity_score[movie_index]))
    
    # Trier les scores de similarité dans l'ordre décroissant (le film le plus similaire en premier)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # # Sélectionner les `num_recommendations` meilleurs scores, en ignorant le premier (c'est le même film)
    similar_movies = similarity_scores[0:num_recommendations]
    similar_movies = [
        (i, score) for i, score in similar_movies if movies.iloc[i]['title'] != exclude_movie
    ]
    # Afficher les recommandations avec les titres des films
    print(f"Films similaires à : {movies.loc[movie_index]['title']}\n")
    for i, score in similar_movies:
        print(f" - {movies.iloc[i]['title']} (similarité : {score:.2f})")
recommend_movies(film1_id, similarity_score, movies, num_recommendations=20, exclude_movie = 'Deadpool 2 (2018)')

def recommend_movies_for_user(user_id, user_similarity_df, user_movie_matrix, num_recommendations=5):
    """
    Recommande des films pour un utilisateur donné en se basant sur des utilisateurs similaires.
    """
    # Trouver les utilisateurs les plus similaires (excluant lui-même)
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id).index

    # Récupérer les notes des utilisateurs similaires
    similar_users_ratings = user_movie_matrix.loc[similar_users]

    # Faire la moyenne des notes pour chaque film (pondérée par la similarité, si besoin)
    avg_ratings = similar_users_ratings.mean()

    # Exclure les films déjà notés par l'utilisateur cible
    user_rated_movies = user_movie_matrix.loc[user_id]
    avg_ratings = avg_ratings[user_rated_movies == 0]

    # Trier les films recommandés
    recommended_movies = avg_ratings.sort_values(ascending=False).head(num_recommendations)

    return recommended_movies

# Exemple de recommandation pour un utilisateur (par exemple, userId = 1)
user_id = 1
recommendations = recommend_movies_for_user(user_id, user_similarity_df, user_movie_matrix)

print(f"Recommandations pour l'utilisateur {user_id} :")
print(recommendations)



Matrice de similarité entre utilisateurs calculée :
userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   

userId       8         9         10   ...       601       602       603  \
userId                                ...                                 
1       0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486   
2       0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997   
3       0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992   
4       0.06

In [22]:
print("The Movies")
movies.iloc[9708]

The Movies


movieId                                 187593
title                        Deadpool 2 (2018)
genres                [Action, Comedy, Sci-Fi]
Action                                       1
Film-Noir                                    0
Children                                     0
IMAX                                         0
Animation                                    0
Romance                                      0
Fantasy                                      0
Mystery                                      0
Drama                                        0
Horror                                       0
Sci-Fi                                       1
(no genres listed)                           0
Musical                                      0
Western                                      0
War                                          0
Documentary                                  0
Adventure                                    0
Comedy                                       1
Crime        

In [23]:
print("Cleaned Ratings dataset:")
genre_data

Cleaned Ratings dataset:


,Action,Film-Noir,Children,IMAX,Animation,Romance,Fantasy,Mystery,Drama,Horror,Sci-Fi,(no genres listed),Musical,Western,War,Documentary,Adventure,Comedy,Crime,Thriller
0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0
1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9736,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
9737,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
9738,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9739,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
